In [1]:
import watertap as wt
import pandas as pd
from pyomo.environ import value, Block
from idaes.core import FlowsheetBlock
import numpy as np
from case_study_trains import *
import case_study_trains
import pyomo.environ as env

m = wt.watertap_setup(dynamic=False)


wt.case_study_trains.train = {"case_study": "upw",
                             "reference": "nawi",
                             "scenario": "baseline"}


wt.case_study_trains.source_water = {"case_study": "upw", 
                             "reference": "nawi",
                             "scenario": "baseline",
                             "water_type": ['upw_brine', 'fab25']}

m = wt.case_study_trains.get_case_study(m=m) # flow is set as case study flow unless defined.


upw

------- Adding Unit Processes -------
brine_ro
treated_storage
onsite_storage
media_filtration
uv_aop
gac_gravity
ultra_filtration
treated_storage_2
media_filtration_2
ultra_filtration_2
gac_gravity_2
reverse_osmosis
treated_storage_3
cooling_supply
industrial
industrial_2
municipal_wwtp
-------------------------------------

adding source
adding source
adding splitter

Connecting unit processes...


In [2]:
wt.display.show_train2(model_name=m)

In [3]:
# RUN MODEL with optimal ro so that the model solves. Then runs again with set pressure. 

In [4]:
# # RUN MODEL with optimal ro --> estimating area and pressure for optimal LCOW
# so that the model solves and gets you results. Then runs again with set pressure.
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).feed.pressure.unfix()
        getattr(m.fs, key).membrane_area.unfix()
        print("Unfixing feed presure and area for", key, '...\n')
        
        
wt.run_water_tap(m=m, objective=True, skip_small=True)

# prints RO results
print("\nOptimal RO area and pressures:")
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(f'\tFeed pressure for {key}: {round(getattr(m.fs, key).feed.pressure[0](), 2)} bar')
        print(f'\tMembrane area for {key}: {round(getattr(m.fs, key).membrane_area[0](), 2)} m2')

# RESET PRESSURE TO USER INPUT
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        if "feed_pressure" in m.fs.pfd_dict[key]["Parameter"]:
            if m.fs.pfd_dict[key]["Parameter"]["type"] == "pass":
                getattr(m.fs, key).feed.pressure.fix(m.fs.pfd_dict[key]["Parameter"]["feed_pressure"])
                print(f'\nSetting feed presure for {key} to {m.fs.pfd_dict[key]["Parameter"]["feed_pressure"]} bar (user input)...\n')

wt.run_water_tap(m=m, objective=True, print_model_results="summary")

for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(f'\tFeed pressure for {key}: {round(getattr(m.fs, key).feed.pressure[0](), 2)} bar')
        print(f'\tMembrane area: for {key}: {round(getattr(m.fs, key).membrane_area[0](), 2)} m2')

Unfixing feed presure and area for brine_ro ...

Unfixing feed presure and area for reverse_osmosis ...

----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------

Optimal RO area and pressures:
	Feed pressure for brine_ro: 31.66 bar
	Membrane area for brine_ro: 6216.65 m2
	Feed pressure for reverse_osmosis: 29.15 bar
	Membrane area for reverse_osmosis: 3256.54 m2

Setting feed presure for brine_ro to 85 bar (user input)...


Setting feed presure for reverse_osmosis to 85 bar (user input)...

----------------------------------------------------------------------

Degrees of Freedom: 2

WaterTAP3 solution optimal 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


BRINE RO:

	total cap investment: 1.61679
	cat and chem cost: 0.01538
	electricity cost: 0.20836
	total fixed op cost: 0.02145

TRE

In [5]:
# If you need the system recovery to match better.... set a maximum recovery rate.

In [6]:
# set cap utilization factor
m.fs.costing_param.plant_cap_utilization = 1

In [7]:
wt.run_water_tap(m=m, objective=True, print_model_results="summary")
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(key, "feed pressure", getattr(m.fs, key).feed.pressure[0]())
        print(key, "membrane area", getattr(m.fs, key).membrane_area[0]())

----------------------------------------------------------------------

Degrees of Freedom: 2

WaterTAP3 solution optimal 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


BRINE RO:

	total cap investment: 1.61679
	cat and chem cost: 0.01538
	electricity cost: 0.20836
	total fixed op cost: 0.02145

TREATED STORAGE:

	total cap investment: 2.15684
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.02835

ONSITE STORAGE:

	total cap investment: 2.79508
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.03695

MEDIA FILTRATION:

	total cap investment: 0.61879
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.00824

UV AOP:

	total cap investment: 3.07383
	cat and chem cost: 0.00667
	electricity cost: 0.00434
	total fixed op cost: 0.04093

GAC GRAVITY:

	total cap investment: 0.7555
	cat and chem cost: 0.08927
	electricity cost: 0.00228
	total fixed op cost: 0.00997

U

In [8]:
# RESET AREA BASED ON RO RECOVERY
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).membrane_area.fix(getattr(m.fs, key).membrane_area[0]())

In [9]:
# Readjust recovery constraint and deactivate objective constraint
m.fs.objective_function.deactivate() # deactivate LCOW objective function

In [10]:
# NOW RUN AS SIMULATION
wt.run_water_tap(m=m, objective=False, print_model_results="summary")

----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


BRINE RO:

	total cap investment: 1.61679
	cat and chem cost: 0.01538
	electricity cost: 0.20836
	total fixed op cost: 0.02145

TREATED STORAGE:

	total cap investment: 2.15684
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.02835

ONSITE STORAGE:

	total cap investment: 2.79508
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.03695

MEDIA FILTRATION:

	total cap investment: 0.61879
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.00824

UV AOP:

	total cap investment: 3.07383
	cat and chem cost: 0.00667
	electricity cost: 0.00434
	total fixed op cost: 0.04093

GAC GRAVITY:

	total cap investment: 0.7555
	cat and chem cost: 0.08927
	electricity cost: 0.00228
	total fixed op cost: 0.00997

U

In [11]:
# creates csv in results folder with the name: *case_study*_*scenario*.csv
# In this case, save the final baseline result.

df = wt.get_results_table(m=m, case_study=wt.case_study_trains.source_water["case_study"], 
                                scenario=wt.case_study_trains.source_water["scenario"])

In [12]:
###### SENSITIVITY ANALYSES 

In [13]:
# Find bounds for RO area:
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        
        stash_value = value(getattr(m.fs, key).membrane_area[0])
        getattr(m.fs, key).membrane_area.unfix()
        m.fs.objective_function1 = env.Objective(expr=getattr(m.fs, key).membrane_area[0] * 1, sense=env.minimize)
        wt.run_water_tap(m = m, objective=False)
        print("LCOW -->", m.fs.costing.LCOW())
        print(key, "Minimum -->", getattr(m.fs, key).membrane_area[0]())

        m.fs.objective_function1 = env.Objective(expr=getattr(m.fs, key).membrane_area[0], sense=env.maximize)
        wt.run_water_tap(m = m, objective=False)
        print("LCOW -->", m.fs.costing.LCOW())
        print(key, "Maximum -->", getattr(m.fs, key).membrane_area[0]())

        getattr(m.fs, key).membrane_area.fix(stash_value)


----------------------------------------------------------------------

Degrees of Freedom: 1

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 1.3849228134815366
brine_ro Minimum --> 10.0
----------------------------------------------------------------------

Degrees of Freedom: 1

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.753580485064619
brine_ro Maximum --> 2494.9714551623674
----------------------------------------------------------------------

Degrees of Freedom: 1

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.9776689825048169
reverse_osmosis Minimum --> 10.0
----------------------------------------------------------------------

Degrees of Freedom: 1

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.7537019166527169
reverse_os

In [14]:
# Find bounds for RO pessure:
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        
        stash_value = value(getattr(m.fs, key).feed.pressure[0])
        getattr(m.fs, key).feed.pressure.unfix()
        m.fs.objective_function1 = env.Objective(expr=getattr(m.fs, key).feed.pressure[0], sense=env.minimize)
        wt.run_water_tap(m = m, objective=False)
        print("LCOW -->", m.fs.costing.LCOW())
        print(key, "Minimum -->", getattr(m.fs, key).feed.pressure[0]())

        m.fs.objective_function1 = env.Objective(expr=getattr(m.fs, key).feed.pressure[0], sense=env.maximize)
        wt.run_water_tap(m = m, objective=False)
        print("LCOW -->", m.fs.costing.LCOW())
        print(key, "Maximum -->", getattr(m.fs, key).feed.pressure[0]())

        getattr(m.fs, key).feed.pressure.fix(stash_value)

----------------------------------------------------------------------

Degrees of Freedom: 1

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 1.157800716233367
brine_ro Minimum --> 5.49493834135894
----------------------------------------------------------------------

Degrees of Freedom: 1

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.7619854625287871
brine_ro Maximum --> 91.67711242352293
----------------------------------------------------------------------

Degrees of Freedom: 1

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.8996869350327613
reverse_osmosis Minimum --> 5.001633868640603
----------------------------------------------------------------------

Degrees of Freedom: 1

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.760

In [15]:
m.fs.objective_function1.deactivate()

In [16]:
# run to rest before sensitivity
wt.run_water_tap(m = m, objective=False)
print("LCOW -->", m.fs.costing.LCOW())

----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.7534830410535733
